# Model for common spatial pattern (CSP) and Riemannian method feature calculation and classification for EEG data


In [1]:
import time
import numpy as np
from tools.csp import generate_projection, generate_eye, extract_feature
from tools.filters import load_filterbank
from sklearn.model_selection import KFold
from sklearn.svm import LinearSVC, SVC
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from tools.data import DreemDatasets
from preprocessing import Compose, ExtractBands, ExtractSpectrum
from models.riemannian_multiscale import riemannian_multiscale

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Config

In [2]:
fs = 50.  # sampling frequency
NO_channels = 7  # number of EEG channels
NO_riem = int(NO_channels * NO_channels + 1) / 2  # Total number of CSP feature per band and timewindow
bw = np.array([2, 4, 8, 13, 22])
ftype = 'butter'  # 'fir', 'butter'
forder = 2  # 4
filter_bank = load_filterbank(bw, fs, order=forder, max_freq=23, ftype=ftype)  # get filterbank coeffs
time_windows_flt = np.array([[0, 30],
                            [5, 25],
                             [15, 30],
                             [10, 25],
                             [5, 20],
                             [5, 15],
                             [0, 10],
                             [5, 15],
                             [15, 25],
                             [10, 20],
                             [5, 15],
                             [5, 10]]) * fs
time_windows = time_windows_flt.astype(int)
# restrict time windows and frequency bands
#time_windows = time_windows[0:1]  # use only largest timewindow

NO_bands = filter_bank.shape[0]
riem_opt = "No_Adaptation"  # {"Riemann","Riemann_Euclid","Whitened_Euclid","No_Adaptation"}
rho = 0.1
NO_csp = 20  # Total number of CSP feature per band and timewindow
useCSP = True


## Generate dataset qui vont biens

On veut : 
    
    1- Tout le train sans split
    2- Tout le train avec split
    3- Tout le train équilibré sans split
    4- Tout le train équilibré avec split
    5- Tout le test 

In [3]:
use_datasets = ['accelerometer_x','accelerometer_y','accelerometer_z','eeg_1', 'eeg_2', 'eeg_3', 'eeg_4', 'eeg_5', 'eeg_6', 'eeg_7', 'pulse_oximeter_infrared']
seed = 1
"""train_set, val_set = DreemDatasets('dataset/train.h5', 'dataset/train_y.csv', 
                                   split_train_val=1, seed=seed, balance_data=False,keep_datasets=use_datasets).get()
train_set.save_data("dataset/all/train")
train_set.close()  # Ne ferme que les fichiers h5. Si mis en mémoire, on a toujours accès aux données !
val_set.close()
train_set, val_set = DreemDatasets('dataset/train.h5', 'dataset/train_y.csv', 
                                   split_train_val=0.8, seed=seed, balance_data=False,keep_datasets=use_datasets).get()
train_set.save_data("dataset/all/train_split")
val_set.save_data("dataset/all/val_split")
train_set.close()  # Ne ferme que les fichiers h5. Si mis en mémoire, on a toujours accès aux données !
val_set.close()
train_set, val_set = DreemDatasets('dataset/train.h5', 'dataset/train_y.csv', 
                                   split_train_val=1, seed=seed, balance_data=True,keep_datasets=use_datasets).get()
train_set.save_data("dataset/balanced/train")
train_set.close()  # Ne ferme que les fichiers h5. Si mis en mémoire, on a toujours accès aux données !
val_set.close()
train_set, val_set = DreemDatasets('dataset/train.h5', 'dataset/train_y.csv', 
                                   split_train_val=0.8, seed=seed, balance_data=True,keep_datasets=use_datasets).get()
train_set.save_data("dataset/balanced/train_split")
val_set.save_data("dataset/balanced/val_split")
train_set.close()  # Ne ferme que les fichiers h5. Si mis en mémoire, on a toujours accès aux données !
val_set.close()"""


from tools.data import DreemDataset
test_set = DreemDataset('dataset/test.h5', keep_datasets=use_datasets).init()
test_set.save_data("dataset/all/test")
test_set.close()



'train_set, val_set = DreemDatasets(\'dataset/train.h5\', \'dataset/train_y.csv\', \n                                   split_train_val=1, seed=seed, balance_data=False,keep_datasets=use_datasets).get()\ntrain_set.save_data("dataset/all/train")\ntrain_set.close()  # Ne ferme que les fichiers h5. Si mis en mémoire, on a toujours accès aux données !\nval_set.close()\ntrain_set, val_set = DreemDatasets(\'dataset/train.h5\', \'dataset/train_y.csv\', \n                                   split_train_val=0.8, seed=seed, balance_data=False,keep_datasets=use_datasets).get()\ntrain_set.save_data("dataset/all/train_split")\nval_set.save_data("dataset/all/val_split")\ntrain_set.close()  # Ne ferme que les fichiers h5. Si mis en mémoire, on a toujours accès aux données !\nval_set.close()\ntrain_set, val_set = DreemDatasets(\'dataset/train.h5\', \'dataset/train_y.csv\', \n                                   split_train_val=1, seed=seed, balance_data=True,keep_datasets=use_datasets).get()\ntrain_set.s

In [4]:
def get_data(path, train= True,  one_vs_all = False, limit= None):
    if train:
        for i in range(7):
            if i==0:
                feature_0 = np.load("dataset/"+path+"/train_split/eeg_" + str(i + 1) + ".npy")
                X = np.zeros((7, feature_0.shape[0], feature_0.shape[1]))
                X[0] = feature_0
                del feature_0
            else:
                X[i] = np.load("dataset/"+path+"/train_split/eeg_" + str(i + 1) + ".npy")
        Y = np.load("dataset/"+path+"/train_split/targets.npy")
        X = X.transpose((1, 0, 2))
    else:
        for i in range(7):
            if i==0:
                feature_0 = np.load("dataset/"+path+"/val_split/eeg_" + str(i + 1) + ".npy")
                X = np.zeros((7, feature_0.shape[0], feature_0.shape[1]))
                X[0] = feature_0
                del feature_0
            else:
                X[i] = np.load("dataset/"+path+"/val_split/eeg_" + str(i + 1) + ".npy")
        Y = np.load("dataset/"+path+"/val_split/targets.npy")
        X = X.transpose((1, 0, 2))
    if one_vs_all:
        Y[Y > 2] = 0
        Y[Y < 2] = 0
        Y[Y == 2] = 1
    if limit is not None:
        X = X[:limit]
        Y = Y[:limit]
    return(X, Y)

#path = "balanced"
#train_data, train_label = get_data(path, train = True, one_vs_all = False, limit= 1000)
#eval_data, eval_label = get_data(path, train = False, one_vs_all = False, limit= 1000)
#print(train_data.shape, train_label.shape)


## Extraction des features par CSP

In [5]:
def get_features(data, label, useCSP = True, NO_csp = 20):
    if useCSP:
        w = generate_projection(data, label, NO_csp, filter_bank, time_windows, NO_classes=5)
    else:
        w = generate_eye(data, label, filter_bank, time_windows)
    feature_mat = extract_feature(data, w, filter_bank, time_windows)
    return(w, feature_mat)

In [12]:
NO_time_windows = int(time_windows.size / 2)
NO_features = NO_csp * NO_bands * NO_time_windows



w, train_feat_CSP = get_features(train_data, train_label, useCSP)



KeyboardInterrupt: 

## Extraction des features par Riemann

In [7]:
NO_time_windows = time_windows.shape[0]
NO_features = NO_riem * NO_bands * NO_time_windows
riemann = riemannian_multiscale(filter_bank, time_windows, riem_opt=riem_opt, rho=rho, vectorized=True)
train_feat_R = riemann.fit(train_data)

## Random Forest

In [8]:
RF_CSP = RandomForestClassifier(n_estimators=1000, random_state=0)
RF_CSP.fit(train_feat_CSP, train_label)

RF_R = RandomForestClassifier(n_estimators=1000, random_state=0)
RF_R.fit(train_feat_R, train_label)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

## Eval

In [9]:
eval_feature_CSP = extract_feature(eval_data, w, filter_bank, time_windows)
eval_feature_R = riemann.features(eval_data)


labels_pred = RF_CSP.predict(eval_feature_CSP)
CM = confusion_matrix(eval_label, labels_pred)
Acc = accuracy_score(eval_label, labels_pred)
F1 = f1_score(eval_label, labels_pred, average='macro')

print(CM, Acc, F1)

labels_pred = RF_R.predict(eval_feature_R)
CM = confusion_matrix(eval_label, labels_pred)
Acc = accuracy_score(eval_label, labels_pred)
F1 = f1_score(eval_label, labels_pred, average='macro')

print(CM, Acc, F1)

[[151  26   4   8  12]
 [ 53  81  23  14  39]
 [ 22  17  81  43  31]
 [ 10   6  24 152   4]
 [ 30  17  25  20 107]] 0.572 0.56199956112291
[[152  25   5   3  16]
 [ 59  79  23   7  42]
 [ 23  17  73  50  31]
 [ 14   6  26 146   4]
 [ 25  33  22  19 100]] 0.55 0.5395647989287642


Il faudrait : 

    1 - Comprendre pk N = 20 marche
    2 - Faire des stats sur les méthodes (temps et accuracy)      ok
    3 - Changer RF par Perceptron
    4 - Ajouter les features des 3 autres courbes + les probas
    5 - Kmeans préliminaire
    6 - Images + Resnet

## Stats

In [6]:
path = "all"
train_data, train_label = get_data(path, train = True, one_vs_all = False)
eval_data, eval_label = get_data(path, train = False, one_vs_all = False)
print(train_data.shape, train_label.shape)


(30631, 7, 1500) (30631,)


In [9]:
useCSP = True
NO_time_windows = int(time_windows.size / 2)
start = time.time()
w, train_feat_CSP = get_features(train_data, train_label, useCSP)
RF_CSP = RandomForestClassifier(n_estimators=1000, random_state=0)
RF_CSP.fit(train_feat_CSP, train_label)
eval_feature_CSP = extract_feature(eval_data, w, filter_bank, time_windows)
np.save("features_CSP_train_split_True.npy", train_feat_CSP)
np.save("features_CSP_val_split_True.npy", eval_feature_CSP)
labels_pred = RF_CSP.predict(eval_feature_CSP)
CM = confusion_matrix(eval_label, labels_pred)
Acc = accuracy_score(eval_label, labels_pred)
F1 = f1_score(eval_label, labels_pred, average='macro')
print(time.time()-start)
print(CM, Acc, F1)

useCSP = False
NO_time_windows = int(time_windows.size / 2)
start = time.time()
w, train_feat_CSP = get_features(train_data, train_label, useCSP)
RF_CSP = RandomForestClassifier(n_estimators=1000, random_state=0)
RF_CSP.fit(train_feat_CSP, train_label)
eval_feature_CSP = extract_feature(eval_data, w, filter_bank, time_windows)
np.save("features_CSP_train_split_False.npy", train_feat_CSP)
np.save("features_CSP_val_split_False.npy", eval_feature_CSP)
labels_pred = RF_CSP.predict(eval_feature_CSP)
CM = confusion_matrix(eval_label, labels_pred)
Acc = accuracy_score(eval_label, labels_pred)
F1 = f1_score(eval_label, labels_pred, average='macro')
print(time.time()-start)
print(CM, Acc, F1)

6050.65074467659
[[ 502    5  165    4   59]
 [  37   13  149    0   61]
 [  53    4 2995   79  306]
 [  15    0  425  722   24]
 [  59    6  546    9 1420]] 0.7380517106294071 0.6062919675553691
2918.735505580902
[[ 464    8  179    4   80]
 [  40   16  133    0   71]
 [  76    7 2902  107  345]
 [  13    0  502  659   12]
 [  53    6  622   17 1342]] 0.7029250457038391 0.5768417304725457


In [10]:
methods = ["No_Adaptation", "Riemann","Riemann_Euclid","Whitened_Euclid","No_Adaptation"]
NO_time_windows = time_windows.shape[0]
for riem_opt in methods:
    try:
        start = time.time()
        NO_time_windows = time_windows.shape[0]
        NO_features = NO_riem * NO_bands * NO_time_windows
        riemann = riemannian_multiscale(filter_bank, time_windows, riem_opt=riem_opt, rho=rho, vectorized=True)
        train_feat_R = riemann.fit(train_data)
        RF_R = RandomForestClassifier(n_estimators=1000, random_state=0)
        RF_R.fit(train_feat_R, train_label)
        eval_feature_R = riemann.features(eval_data)
        np.save("features_R_train_split_"+str(riem_opt), train_feat_CSP)
        np.save("features_R_val_split_"+str(riem_opt), eval_feature_CSP)
        labels_pred = RF_R.predict(eval_feature_R)
        CM = confusion_matrix(eval_label, labels_pred)
        Acc = accuracy_score(eval_label, labels_pred)
        F1 = f1_score(eval_label, labels_pred, average='macro')
        print(time.time()-start)
        print(CM, Acc, F1)
    except Exception as e:
        print(e)
        pass
    

5549.880460977554
[[ 480    5  179    0   71]
 [  44   12  140    0   64]
 [  55    6 2934  110  332]
 [  17    1  499  657   12]
 [  46    8  598   18 1370]] 0.7120658135283364 0.5808282267416092


/usr/local/lib/python3.6/dist-packages/pyriemann/utils/base.py:12: RuntimeWarning: invalid value encountered in log
  eigvals = numpy.diag(operator(eigvals))


Covariance matrices must be positive definite. Add regularization to avoid this error.
Input contains NaN, infinity or a value too large for dtype('float32').
4730.435000419617
[[ 480    2  196    5   52]
 [  48    6  159    3   44]
 [  52    3 3021   79  282]
 [  11    0  541  567   67]
 [  57    3  745   38 1197]] 0.6882998171846435 0.5475961588532392
5817.639447450638
[[ 480    5  179    0   71]
 [  44   12  140    0   64]
 [  55    6 2934  110  332]
 [  17    1  499  657   12]
 [  46    8  598   18 1370]] 0.7120658135283364 0.5808282267416092


RF - Boosting - NN

In [2]:
from preprocessing.features import ExtractFeatures
extract_features = ExtractFeatures(bands='*', features=['min', 'max', 'energy', 'frequency'])

transformations = {
    "XXX": extract_features,
    "XXX": extract_features,
    "XXX": extract_features
}

train_set, val_set = DreemDatasets('dataset/train.h5', 'dataset/train_y.csv', 
                                   split_train_val=0.8, seed=seed, keep_datasets=use_datasets,
                                   transforms=dataset_transform_spectrum).get()